# M1M3 position data
Craig Lage - 16-Apr-23 \
Comparing different position measurements.

In [ ]:
import sys, time, os, asyncio, glob
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import pickle as pkl
from astropy.time import Time, TimeDelta
import lsst.ts.cRIOpy.M1M3FATable as M1M3FATable
from scipy.interpolate import UnivariateSpline
from lsst.summit.utils.tmaUtils import TMAEventMaker, TMAState, getSlewsFromEventList
from lsst.summit.utils.efdUtils import (makeEfdClient,
                                        getEfdData,
                                        getMostRecentRowWithDataBefore,
                                        efdTimestampToAstropy,
                                       )

In [ ]:
FATABLE = M1M3FATable.FATABLE

In [ ]:
client = makeEfdClient()
dayObs = 20230731
#dayObs = 20230601
eventMaker = TMAEventMaker()
events = eventMaker.getEvents(dayObs)
slews = [e for e in events if e.type==TMAState.SLEWING]
tracks = [e for e in events if e.type==TMAState.TRACKING]
print(f'Found {len(slews)} slews and {len(tracks)} tracks')

In [ ]:
data = {}
for dayObs in range(20230710, 20230731, 1):
    els = []
    xPos = []
    yPos = []
    zPos = []
    xRot = []
    yRot = []
    zRot = []

    events = eventMaker.getEvents(dayObs)
    slews = [e for e in events if e.type==TMAState.SLEWING]
    print(f'{dayObs}:found {len(slews)} slews ')
    if len(slews) < 1:
        continue

    for slew in slews:
        dT = 5.0
        start = slew.end-TimeDelta(dT, format='sec')
        end = slew.end+TimeDelta(dT, format='sec')
        ims_data = await client.select_time_series("lsst.sal.MTM1M3.imsData", "*", start, end)
        az = await client.select_time_series("lsst.sal.MTMount.azimuth","actualPosition", start, end)
        el = await client.select_time_series("lsst.sal.MTMount.elevation","actualPosition", start, end)
        try:
            zPosFinal = abs(ims_data['zPosition'].values[-1])
            #print(el.values[-1], zPosFinal)
            if zPosFinal > 0.004: # Test if mirror is raised
                dayObss.append(dayObs)
                els.append(el.values[-1][0])
                xPos.append(ims_data['xPosition'].values[-1] * 1.0E6)
                yPos.append(ims_data['yPosition'].values[-1] * 1.0E6)
                zPos.append(ims_data['zPosition'].values[-1] * 1.0E6)
                xRot.append(ims_data['xRotation'].values[-1] * 1.0E6)
                yRot.append(ims_data['yRotation'].values[-1] * 1.0E6)
                zRot.append(ims_data['zRotation'].values[-1] * 1.0E6)
        except:
            continue
    if len(els) > 10:
        data[dayObs] = [els, [xPos, yPos, zPos, xRot, yRot, zRot]]


In [ ]:
len(data)

In [ ]:
data.keys()

In [ ]:
data[20230710]

In [ ]:
ims_data['zPosition'].plot()

In [ ]:
%matplotlib inline
names = ['xPosition', 'yPosition', 'zPosition', 'xRotation', 'yRotation', 'zRotation']
limits = [6.0, 6.0, 1.0, 24.0, 24.0, 24.0]

fig, axs = plt.subplots(3,2,figsize=(8,10))
plt.subplots_adjust(hspace=0.5, wspace=0.5)
plt.suptitle(f"Final mirror position after a slew", fontsize=16)
plot_counter = 0
for i in range(3):
    for j in range(2):
        if plot_counter > 2:
            unit = 'microDeg'
        else:
            unit = 'um'
        for dayObs in data.keys():
            [els, plots] = data[dayObs] 

            axs[i][j].set_title(names[plot_counter])
            axs[i][j].set_xlim(0, 90)
            axs[i][j].set_xlabel("Elevation (degrees)")
            axs[i][j].set_ylabel(f'IMS ({unit})')
            axs[i][j].scatter(els, plots[plot_counter], label=dayObs)
        plot_counter += 1
axs[0][1].legend()
plt.savefig(f"/home/c/cslage/u/MTM1M3/data/Final_Mirror_Position_02Aug23.png")